---
## **主題: 記錄每天攝取的食物，統計當天吃的熱量及營養素**

---
## Import toolkit

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interact_manual  #互動套件
import ipywidgets as widgets  #interact控制元件
import requests
from bs4 import BeautifulSoup as bs
from lxml import html
from IPython import display
from matplotlib.font_manager import FontProperties
myf = FontProperties(fname='C:\Windows\Fonts\msjh.ttc')
plt.rcParams['font.sans-serif']=['SimHei']

---
## Function  definition 1 -- search4nutrient()
- 此函式用來找給定食物的熱量及營養素
- Parameter為欲查詢熱量的'食物名稱'


In [2]:
def search4nutrient(name='白飯'):
    # 設定營養資料查詢網頁的rul
    url = 'https://www.cfs.gov.hk/tc_chi/nutrient/search3.php'
    # 送post必需的一些資料
    payload = {
        "keyword":name,
        "keylang": "C",
        "inShortMode": "0",
        "searchcriteria": "and",
        "keyword2": ""
    }
    h = requests.post(url, data=payload)  # 開始送post給網站
    soup = bs(h.text, 'html.parser')  # beautifulsoup解析成html形式
    a = soup.select('.colorTable1 > tr > td > ul > li > a')  # 找到對應食物的href,將使用參數進入食物營養素查詢
    # 取第一個結果:
    if(len(a) > 0):
        a = a[0].attrs['href'].replace('javascript:tosubmit (','')
        a = a.replace(');','').replace("'",'').replace(' ','')
        a = a.split(',')

    # 設定該食物熱量的網址及參數(在網站F12裡找)
    url = 'https://www.cfs.gov.hk/tc_chi/nutrient/fc-myreport.php'
    payload = {
        "fg_id": a[0],
        "fsg_id": a[1],
        "food_id": a[2],
        "inShortMode": ""
    }
    h = requests.post(url, data=payload)  # 開始送post給網站
    soup = bs(h.text, 'html.parser')  # beautifulsoup解析成html形式
    tb = soup.find_all('table')[1]  # 找到第2個table(紀錄熱量的table)
    pf = pd.read_html(str(tb))[0]  # 用read_html直接把該table的html格式轉成dataFrame
    # 以下幾步為處理dataFrame的過程，不多贅述
    pf = pf[(pf['營養']!='主要成分')]
    pf = pf[(pf['營養']!='脂質')]
    pf = pf[(pf['營養']!='礦物質和維他命')]
    pf['數值'].fillna(value=0, inplace=True)
    pf.reset_index(inplace=True, drop=True)
    for i in range(len(pf['營養'])):
        pf['營養'][i] = pf['營養'][i].replace('*','').replace(' ','')
    pf.index = [pf['營養'][i] for i in range(len(pf['營養']))]
    pf.drop('營養',axis=1,inplace=True)
    pf = pf[['數值','單位']]
    pf;
    return pf

search4nutrient()

,數值,單位
能量,130,千卡
蛋白質,2.38,克
碳水化合物,28.59,克
脂肪,0.21,克
膳食纖維,0,克
糖,0,克
飽和脂肪,0.057,克
反式脂肪,0,克
膽固醇,0,毫克
鈣,3,毫克


---
## Function definition 2
- **RecordEat(Date, item, gram, note)**
    - 紀錄吃的東西、重量...

In [3]:
# Date 是屬於datetime.date object, 用Date.strftime('%Y(或%m,%d)')來得到年月日
def RecordEat(Date, item, gram, note):
    # 如果輸入不完整，則不紀錄這筆資料
    if(len(item)==0 or len(gram)==0 or (Date==None)):
        print("請輸入完整內容")
        return
    data = [[Date, item, gram, note]]  # 建list，這裡輸出依序是'日期', '品項', '金額', '備註'
    df = pd.DataFrame(data, columns = ['日期', '食物名稱', '重量(公克)', '備註'])  # 存csv的標準格式:'DataFrame'
    # 如果csv已經存在，則直接寫資料，不寫header
    # 如果不存在，則把header也一起寫進csv檔
    try:
        with open('eatlist.csv','r') as f:
            True
        df.to_csv('eatlist.csv',mode='a', header=False, index=False)
    except:
        df.to_csv('eatlist.csv',mode='w', header=True, index=False)
    print(' '*np.random.randint(10)+'紀錄成功！')
    return

---
## Function definition 3
- **CalculEat(Date)**
    - 秀出特定日期當天攝取的食物熱量、營養素等等

In [4]:
def CalculEat(Date):
    TotalCalo = {'能量':[0,'千卡'], '蛋白質':[0,'克'], '碳水化合物':[0,'克'], '脂肪':[0,'克'], 
                 '膳食纖維':[0,'克'], '糖':[0,'克'], '飽和脂肪':[0,'克'], '反式脂肪':[0,'克'], 
                 '膽固醇':[0,'毫克'], '鈣':[0,'毫克'], '銅':[0,'毫克'], '鐵':[0,'毫克'], 
                 '鎂':[0,'毫克'], '錳':[0,'毫克'], '磷':[0,'毫克'], '鉀':[0,'毫克'], 
                 '鈉':[0,'毫克'], '鋅':[0,'毫克'], '維他命C':[0,'毫克']}
    # try open csv
    try:
        df = pd.read_csv('eatlist.csv')
    except:
        print('目前還沒有任何紀錄可查詢，請先至少記錄一筆資料後再查詢。')
        return
    # 找出日期一樣的
    df2 = df[df['日期']==str(Date)]
    if(len(df2)==0):
        print(Date.strftime('%Y') + '/' + Date.strftime('%m') + '/' + Date.strftime('%d') + "沒有紀錄");
        return
    df2.reset_index(inplace=True, drop=True)
    # 統計當天總熱量及養分
    for i in range(len(df2['食物名稱'])):
        food = str(df2['食物名稱'][i])
        calo = search4nutrient(food)
        for key, value in TotalCalo.items():
            TotalCalo[key][0] += float(calo['數值'][key])*(float(df2[df2['食物名稱']==food]['重量(公克)'])/100.0)
    todf = pd.DataFrame.from_dict(TotalCalo,orient='index',columns=['數值','單位'])
    todf.sort_values(by='單位',inplace=True)
    # 用plt畫表格
    plt.figure(1);
    fig, ax = plt.subplots();
    # 隱藏坐標軸
    ax.xaxis.set_visible(False);
    ax.yaxis.set_visible(False);
    ax.axis('off');
    ax.axis('tight');  # 設置為緊密貼合
    ax.table(cellText=[[round(v[0],2),v[1]] for _,v in TotalCalo.items()], colLabels=todf.columns, rowLabels = [k for k,_ in TotalCalo.items()], loc='center');
    fig.tight_layout();

    # 統計三大營養素圓餅圖
    col = ['醣類','脂質','蛋白質']
    val = [(TotalCalo['糖'][0]+TotalCalo['碳水化合物'][0])*4.0, 
           (TotalCalo['脂肪'][0]+TotalCalo['飽和脂肪'][0]+TotalCalo['反式脂肪'][0])*9.0,
          TotalCalo['蛋白質'][0]*4.0]
    # 畫圓餅圖
    plt.figure(2);
    fig, ax = plt.subplots();
    ax.pie(val, labels = col, autopct='%1.1f%%',shadow=True, startangle=90);
    plt.title('三大營養素攝取比例');
    fig.tight_layout();

---
## 建構interact元件
- 根據RecordEat()建一個互動式表單
- 根據CalculEar()建一個日期選單，讓使用者選擇要查看哪一天

In [5]:
print(' '*16+'每日食物攝取紀錄表')
interact_manual(
    RecordEat,
    Date=widgets.DatePicker(description='日期'),  # widget list DataPicker: 顯示可選擇日期的輸入框
    item=widgets.Text(description='食物名稱',disabled=False,  #widget list Text: 輸入文字
        placeholder='ex:白飯、油麵...'), 
    gram=widgets.Text(description='重量(公克)',disabled=False,
        placeholder='ex:100'), 
    note=widgets.Textarea(description='備註', disabled=False,  #widget list TextArea: 輸入多行文字
        placeholder='ex:九折商品...'),
);


print('\n'*3+' '*16+'查詢當天攝取熱量')
interact_manual(
    CalculEat,
    Date = widgets.DatePicker(description='日期')
);

                每日食物攝取紀錄表


interactive(children=(DatePicker(value=None, description='日期'), Text(value='', description='食物名稱', placeholder…




                查詢當天攝取熱量


interactive(children=(DatePicker(value=None, description='日期'), Button(description='Run Interact', style=Butto…